# Quantum Maze Solver

In [ ]:
import qiskit
from qiskit import QuantumCircuit
import numpy as np
from typing import TypeAlias
from qiskit.circuit.library import GroverOperator
from maze_oracle import MazeOracleGenerator

In [ ]:
Edge: TypeAlias = tuple[int, int]
Path: TypeAlias = list[Edge]

class Maze:
    def __init__(self, num_nodes: int, edges: list[Edge], directed: bool = False):
        self.num_nodes = num_nodes
        self.edges = edges
        self.directed = directed
        self.max_path_length = num_nodes

class QuantumMazeCircuit(Maze, QuantumCircuit):
    def __init__(self, from_maze: Maze):
        self.node_bits_size = int(np.ceil( np.log2(from_maze.num_nodes) ))
        self.num_qubit_in_path = (from_maze.max_path_length + 1) * self.node_bits_size
        self.total_size = self.num_qubit_in_path + from_maze.max_path_length

        Maze.__init__(self, from_maze.num_nodes, from_maze.edges, from_maze.directed)
        QuantumCircuit.__init__(self, self.total_size)

class QuantumMazeSolver:
    def generate_circuit(self, maze: Maze) -> QuantumMazeCircuit:
        ### Setup
        circ = QuantumMazeCircuit(maze)
        circ.name = 'Maze Solver'
        # Get oracle
        oracle = MazeOracleGenerator(circ.num_nodes, circ.max_path_length, circ.directed).add_edges(circ.edges).generate()
        # Get grover's diffuser
        diffuser = GroverOperator(QuantumCircuit(circ.num_qubit_in_path))
        # Compute repetition time
        iterations = int(np.round( (np.pi / 4) * np.sqrt(circ.num_qubit_in_path) ))

        ### Build
        # Make superpositions
        for i in range(circ.num_qubit_in_path):
            circ.h(i)
        # Repeat grover operation
        for i in range(iterations):
            circ.barrier()
            circ.append(oracle, range(circ.total_size))             # Apply oracle
            circ.append(diffuser, range(circ.num_qubit_in_path))    # Apply diffuser

        return circ

    def run(self, circuit: QuantumMazeCircuit, runs: int = 1) -> Path:
        pass

    def solve(self, maze: Maze) -> Path:
        maze_circ = self.generate_circuit(maze)
        path = self.run(maze_circ)
        return path